### Importing necessary libraries

In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine, text

### Making an API request to get the daily data on IBM stocks for the last 20+ years

In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=8F0QJ66VEZNV8T3L&outputsize=full'
r = requests.get(url)
data = r.json()

### Working around JSON formatting

In [3]:
final_data = []
for time, data in data["Time Series (Daily)"].items():
    observation_dict = {
        "timestamp" : time,
        "open" : float(data["1. open"]),
        "high" : float(data["2. high"]),
        "low" : float(data["3. low"]),
        "close" : float(data["5. adjusted close"])
    }
    final_data.append(observation_dict)

NOTE: The next cell is the easier, more pythonic way of doing this step, but iterating allows for real-time manipulation like for example type conversion we see here.

Yavor's contribution to my silly from-scratch solution. (Which has benefits listed above)

In [4]:
#df = pd.DataFrame(data["Time Series (Daily)"])
#df = df.transpose()
# ... conversion ...

### Creating a DataFrame object to load as an SQL table later

NOTE: Converting the 'timestamp' column to a 'datetime' format.

In [5]:
df = pd.DataFrame(final_data)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,timestamp,open,high,low,close
0,2023-02-08,135.71,136.74,135.16,135.980000
1,2023-02-07,135.67,136.40,134.45,135.840000
2,2023-02-06,135.83,136.32,134.95,136.180000
3,2023-02-03,136.35,136.95,135.53,136.940000
4,2023-02-02,135.96,136.72,134.85,136.390000
...,...,...,...,...,...
5851,1999-11-05,92.75,92.94,90.19,49.728782
5852,1999-11-04,94.44,94.44,90.00,50.450607
5853,1999-11-03,95.87,95.94,93.50,51.998949
5854,1999-11-02,96.75,96.81,93.69,52.241394


### Connecting to a TimescaleDB instance

In [6]:
pg_user = 'postgres'
pg_password = 'a5TKQ7SSVGFuQIz5'
pg_hostname = 'primeds.protal.biz:30308'
pg_db_name = 'postgres'

pg_engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_hostname}/{pg_db_name}')
pg_conn = pg_engine.connect()

### Creating a table that will hold the data and converting it into a hypertable which offers better performance

In [7]:
table_query = '''CREATE TABLE IF NOT EXISTS staging.timetest2 (
  timestamp    TIMESTAMPTZ       NOT NULL,
  open         DOUBLE PRECISION  NULL,
  high         DOUBLE PRECISION  NULL,
  low          DOUBLE PRECISION  NULL,
  close        DOUBLE PRECISION  NULL
);'''
result = pg_conn.execute(text(table_query))

In [8]:
hypertable_query = '''SELECT create_hypertable('staging.timetest2', 'timestamp');'''
result = pg_conn.execute(text(hypertable_query))

### Loading the data into a TimescaleDB table

In [9]:
df.to_sql("timetest2", pg_conn, index=False, if_exists='replace', schema='staging')
pg_conn.commit()
pg_conn.close()

762 ms ± 40.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Load function time

CPU times: total: 46.9 ms
Wall time: 773 ms

### Whole cell time

762 ms ± 40.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)